PART ONE ANSWERS:

/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name FROM Facilities WHERE membercost != 0;

/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(name) FROM Facilities WHERE membercost > 0;

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost != 0 AND (membercost < monthlymaintenance * 0.2);

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

(SELECT *
FROM Facilities 
	WHERE facid=1)
UNION
(SELECT *
 FROM Facilities
 	WHERE facid=5);


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name,
	CASE WHEN monthlymaintenance < 100 THEN 'cheap'
		ELSE 'expensive' END AS cost
FROM Facilities;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname, joindate
FROM Members
WHERE joindate = 
	(SELECT MAX(joindate)
	 FROM Members);

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT CONCAT(m.firstname, ' ', m.surname) AS member, f.name AS facility
FROM Bookings AS b
INNER JOIN Members AS m
ON b.memid = m.memid
INNER JOIN Facilities as f
ON b.facid = f.facid
WHERE f.name LIKE '%Tennis Court%'
ORDER BY member;

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT CONCAT(m.firstname, ' ', m.surname) AS member, f.name AS facility,
	CASE WHEN m.memid = 0 THEN b.slots*f.guestcost ELSE b.slots*f.membercost
	END AS cost
FROM Members as m
	INNER JOIN Bookings AS b
	ON m.memid = b.memid
	INNER JOIN Facilities AS f
	ON b.facid = f.facid
WHERE (b.starttime BETWEEN '2012-09-14' and '2012-09-15')
	AND (
	  (m.memid = 0 AND b.slots*f.guestcost > 30) OR
	  (m.memid != 0 AND b.slots*f.membercost >30)
	  )
ORDER BY cost DESC;

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT member, facility, cost
FROM
	(SELECT f.name AS facility, concat(m.firstname, ' ', m.surname) AS member, 
	 CASE
	 		WHEN m.memid = 0 THEN
	 				b.slots*f.guestcost
	 		ELSE
	 				b.slots*f.membercost
	END AS cost
	FROM Members AS m
	INNER JOIN Bookings AS b
	ON m.memid = b.memid
	INNER JOIN Facilities AS f
	ON b.facid = f.facid
	WHERE b.starttime BETWEEN '2012-09-14' AND '2012-09-15') AS bookings
	WHERE cost > 30
ORDER BY cost DESC;

In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('./sqlite_db_pythonsqlite.db')

In [4]:
cur = conn.cursor()

PART 2

In [5]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!

cur.execute("""SELECT name, revenue FROM
  (SELECT f.name, sum(CASE
                    WHEN memid = 0 THEN slots * f.guestcost
                    ELSE slots * f.membercost
                    END) AS revenue
        FROM Bookings AS b
        INNER JOIN Facilities as f
            ON b.facid = f.facid
        GROUP BY f.name
    ) AS agg WHERE revenue < 1000
ORDER BY revenue;""");

In [6]:
res = cur.fetchall()
print(res)

[('Table Tennis', 180), ('Snooker Table', 240), ('Pool Table', 270)]


In [7]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

cur.execute("""SELECT m.firstname AS memfname, m.surname AS memsname, r.firstname AS recfname, r.surname AS recsname
    FROM Members AS m
    LEFT JOIN Members AS r
    ON m.recommendedby = r.memid
ORDER BY m.surname, m.firstname;""");

In [8]:
res = cur.fetchall()
print(res)

[('Florence', 'Bader', 'Ponder', 'Stibbons'), ('Anne', 'Baker', 'Ponder', 'Stibbons'), ('Timothy', 'Baker', 'Jemima', 'Farrell'), ('Tim', 'Boothe', 'Tim', 'Rownam'), ('Gerald', 'Butters', 'Darren', 'Smith'), ('Joan', 'Coplin', 'Timothy', 'Baker'), ('Erica', 'Crumpet', 'Tracy', 'Smith'), ('Nancy', 'Dare', 'Janice', 'Joplette'), ('David', 'Farrell', None, None), ('Jemima', 'Farrell', None, None), ('GUEST', 'GUEST', None, None), ('Matthew', 'Genting', 'Gerald', 'Butters'), ('John', 'Hunt', 'Millicent', 'Purview'), ('David', 'Jones', 'Janice', 'Joplette'), ('Douglas', 'Jones', 'David', 'Jones'), ('Janice', 'Joplette', 'Darren', 'Smith'), ('Anna', 'Mackenzie', 'Darren', 'Smith'), ('Charles', 'Owen', 'Darren', 'Smith'), ('David', 'Pinker', 'Jemima', 'Farrell'), ('Millicent', 'Purview', 'Tracy', 'Smith'), ('Tim', 'Rownam', None, None), ('Henrietta', 'Rumney', 'Matthew', 'Genting'), ('Ramnaresh', 'Sarwin', 'Florence', 'Bader'), ('Darren', 'Smith', None, None), ('Darren', 'Smith', None, None), 

In [9]:
# Q12: Find the facilities with their usage by member, but not guests

cur.execute("""SELECT m.surname AS member, b.facid, SUM(slots) AS "Slots"
    FROM Bookings AS b
    INNER JOIN Members AS m
    ON b.memid = m.memid
    WHERE b.memid != 0
    GROUP BY b.facid
ORDER BY b.facid;""");

In [10]:
res = cur.fetchall()
print(res)

[('Smith', 0, 957), ('Rownam', 1, 882), ('Smith', 2, 1086), ('Smith', 3, 794), ('Smith', 4, 884), ('Joplette', 5, 54), ('Smith', 6, 418), ('Smith', 7, 860), ('Smith', 8, 856)]


In [13]:
# Q13: Find the facilities usage by month, but not guests

cur.execute("""select strftime('%M/%Y', starttime) as month, count(*)
    from Bookings
    group by month
    order by month""");

In [15]:
res1 = cur.fetchall()
print(res1)

[]
